In [1]:
import numpy as np
import pandas as pd

import pickle
import random
import os
import time
from datetime import datetime
from pathlib import Path
import glob
import json
from tqdm.notebook import tqdm

from multiprocessing import Pool, cpu_count
from post_multi import motion_multi, fs_multi

import matplotlib.path as mpltPath
from scipy.spatial import distance
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from scipy.ndimage import median_filter

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from PIL import Image
#%matplotlib inline
plotly.offline.init_notebook_mode(connected=True)
sns.set_style('darkgrid')

from xyz10.io_f_mod import read_data_file
from xyz10.visualize_f_mod import visualize_trajectory, save_figure_to_image
from xyz10.compute_f_mod import compute_step_positions, split_ts_seq, correct_positions
from xyz10.compute_f_mod import compute_steps, compute_headings, compute_stride_length, compute_step_heading, compute_rel_positions
from xyz10.compute_f_mod import correct_positions_mod, correct_positions_mod2, compute_step_positions_mod, compute_step_positions_mod2, split_ts_seq_mod

from xyz10.io_f_mod import read_data_file
from xyz10.visualize_f_mod import visualize_trajectory, save_figure_to_image

FLOOR_NUM_to_ID = {'5a0546857ecc773753327266': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5c3c44b80379370013e0fd2b': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5d27075f03f801723c2e360f': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'},
                 '5d27096c03f801723c31e5e0': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6'},
                 '5d27097f03f801723c320d97': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5d27099f03f801723c32511d': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5d2709a003f801723c3251bf': {0: '1F', 1: '2F', 2: '3F', 3: '4F'},
                 '5d2709b303f801723c327472': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F', 3: '4F'},
                 '5d2709bb03f801723c32852c': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5d2709c303f801723c3299ee': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F', 3: '4F', 4: '5F', 5: '6F', 6: '7F', 7: '8F', 8: '9F'},
                 '5d2709d403f801723c32bd39': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F'},
                 '5d2709e003f801723c32d896': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5da138274db8ce0c98bbd3d2': {0: 'F1', 1: 'F2', 2: 'F3'},
                 '5da1382d4db8ce0c98bbe92e': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5da138314db8ce0c98bbf3a0': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3'},
                 '5da138364db8ce0c98bc00f1': {0: 'F1', 1: 'F2', 2: 'F3'},
                 '5da1383b4db8ce0c98bc11ab': {0: 'F1', 1: 'F2', 2: 'F3'},
                 '5da138754db8ce0c98bca82f': {0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5da138764db8ce0c98bcaa46': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5da1389e4db8ce0c98bd0547': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5da138b74db8ce0c98bd4774': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5da958dd46f8266d0737457b': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'},
                 '5dbc1d84c1eb61796cf7c010': {-1: 'B1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7', 7: 'F8'},
                 '5dc8cea7659e181adb076a3f': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'}}

---------------------------------------------------------

In [25]:
import glob, os
paths = glob.glob('./img_out/predictions/blend_post_x12_1705/*/*median.png')
_ = [os.remove(path) for path in paths]

Function: Blend, Plot_Traces, Snap2Grid, Make_Submit

In [16]:
def blend_predictions(blend_folder, reference_file, is_mean=False):

    blend_paths = glob.glob(blend_folder + "*")
    blend_data = []
    for b_path in blend_paths:
        blend_data.append(pickle.load(open(b_path, "rb")))
        #break
    reference_data = pickle.load(open(blend_folder+reference_file, "rb"))

    compound= {}
    for site_id in reference_data.keys():

        compound[site_id] = {}
        for trace_id in reference_data[site_id].keys():

            _timestamps = reference_data[site_id][trace_id].to_numpy()[:, 3]
            _floor = int(np.median(reference_data[site_id][trace_id].to_numpy()[:, 2]))
            _x = []
            _y = []
            for data in blend_data:

                predicted_record = data[site_id][trace_id].to_numpy()
                _x.append(predicted_record[:, 0])
                _y.append(predicted_record[:, 1])
            
            if is_mean:
                compound[site_id][trace_id] = pd.DataFrame({"x": np.mean(_x, axis=0), "y": np.mean(_y, axis=0), "floor": _floor, "time": _timestamps})
            else:
                compound[site_id][trace_id] = pd.DataFrame({"x": np.median(_x, axis=0), "y": np.median(_y, axis=0), "floor": _floor, "time": _timestamps})

            #break 
    return compound

def blend_predictions_x(blend_folder, reference_file, grid_data, is_mean=False):
    snap_range = 0.2

    blend_paths = glob.glob(blend_folder + "*")
    blend_data = []
    for b_path in blend_paths:
        blend_data.append(pickle.load(open(b_path, "rb")))
        #break
    reference_data = pickle.load(open(blend_folder+reference_file, "rb"))

    compound= {}
    for site_id in tqdm(reference_data.keys()):

        compound[site_id] = {}
        for trace_id in reference_data[site_id].keys():

            _timestamps = reference_data[site_id][trace_id].to_numpy()[:, 3]
            _floor = int(np.median(reference_data[site_id][trace_id].to_numpy()[:, 2]))
            _x = []
            _y = []
            _loss = []
            for data in blend_data:
                predicted_record = data[site_id][trace_id].to_numpy()
                grid_record = grid_data[site_id][_floor]
                _xs = predicted_record[:, 0]
                _ys = predicted_record[:, 1]
                
                _x.append(_xs)
                _y.append(_ys)
                
                #########################  SNAP POINTS + LOSS   ##################################
                trajectory_extent = ((_xs.max() - _xs.min()) ** 2 + (_ys.max() - _ys.min()) ** 2) ** 0.5
                snap_distance = snap_range * trajectory_extent
                
                try:
                    snap_points = []
                    for i_step, _ in enumerate(_xs):
                        _distances = ((grid_record[:, 0] - _xs[i_step]) ** 2 + (grid_record[:, 1] - _ys[i_step]) ** 2) ** 0.5
                        _snap_points = grid_record[_distances <= snap_distance]

                        if len(_snap_points) > 0:
                            snap_points.append(_snap_points)
                except:
                    print(_ys)

                snap_points = np.concatenate(snap_points)
                snap_points = np.array(list(set(map(tuple, snap_points))))

                distances2 = (snap_points[:, 0].reshape(-1, 1) - _xs.reshape(1, -1)) ** 2 + (snap_points[:, 1].reshape(-1, 1) - _ys.reshape(1, -1)) ** 2
                
                _loss.append(np.sum(np.amin(distances2, axis=0)))
                
            ###########################################################
            loss_arr = np.array(_loss)
            best_inds = list(np.where(loss_arr <= np.median(loss_arr))[0])
            _xb = [_x[el] for el in best_inds]
            _yb = [_y[el] for el in best_inds]
            ############################################################
            
            if len(best_inds) > 1:
                if is_mean:
                    compound[site_id][trace_id] = pd.DataFrame({"x": np.mean(_xb, axis=0), "y": np.mean(_yb, axis=0), "floor": _floor, "time": _timestamps})
                else:
                    compound[site_id][trace_id] = pd.DataFrame({"x": np.median(_xb, axis=0), "y": np.median(_yb, axis=0), "floor": _floor, "time": _timestamps})
            else:
                compound[site_id][trace_id] = pd.DataFrame({"x": np.array(_xb), "y": np.array(_yb), "floor": _floor, "time": _timestamps})

            #break 
    return compound

In [3]:
def make_submission(model_name, data, sufix="coarse"):
    
    sample_submit = pd.read_csv("./submit/sample_submission.csv")
    splits = sample_submit.site_path_timestamp.str.split(pat="_", expand=True)
    sub_data = sample_submit.copy(deep=True).join(splits)
    sub_data.rename(columns={0:"site", 1:"path", 2:"timestamp"}, inplace=True)

    gr = sub_data.groupby("path")
    for trace_id in gr.groups:
        timestamps = sub_data.loc[gr.groups[trace_id]].timestamp.to_list()

        site_id = sub_data.loc[gr.groups[trace_id]].site.to_list()[0]
        predicted_record = data[site_id][trace_id].to_numpy()

        func_x = interp1d(predicted_record[:, 3], predicted_record[:, 0], kind="linear", copy=False, fill_value="extrapolate")
        func_y = interp1d(predicted_record[:, 3], predicted_record[:, 1], kind="linear", copy=False, fill_value="extrapolate")

        sub_data.loc[gr.groups[trace_id], "x"] = func_x(timestamps)
        sub_data.loc[gr.groups[trace_id], "y"] = func_y(timestamps)
        sub_data.loc[gr.groups[trace_id], "floor"] = predicted_record[0, 2]
        #break

    _ = [sub_data.pop(col) for col in ["site", "path", "timestamp"]]

    sub_data.to_csv(f"./submit/{model_name}_{sufix}.csv", index=False)
    
def plot_predictions_multi(model_name, data, sufix="coarse", delay_suffix=False):
    
    def swap_trace_floor(predicted_data):
        swap = {}

        for site_id in predicted_data.keys():

            swap[site_id] = {}
            for trace_id in predicted_data[site_id].keys():

                floor_id = predicted_data[site_id][trace_id].floor[0]
                if floor_id not in swap[site_id].keys():
                    swap[site_id][floor_id] = {}
                swap[site_id][floor_id][trace_id] = predicted_data[site_id][trace_id]

        return swap

    data = swap_trace_floor(data)

    try:
        os.makedirs(f"./img_out/predictions/{model_name}/")
    except:
        pass

    n_s = 0
    for site_id in tqdm(data.keys()):  # over sites 
        n_s += 1
        #print(f"Processing Trajectories #{n_s}: Site-{site_id} with {len(data[site_id])} traces")

        try:
            os.makedirs(f"./img_out/predictions/{model_name}/{site_id}")
        except:
            pass

        for floor_id in data[site_id]:  # over traces
            site_path = "./data_in/metadata/" + site_id + "/"
            
            positions = []
            legends = []
            for trace_id in data[site_id][floor_id].keys():
                positions.append(data[site_id][floor_id][trace_id].to_numpy()[:, :2])
                
                if delay_suffix:
                    legends.append(f"{trace_id}_s{int(0.001*data[site_id][floor_id][trace_id].to_numpy()[0, 4])}_e{int(0.001*data[site_id][floor_id][trace_id].to_numpy()[1, 4])}")
                else:
                    legends.append(trace_id)

            try:
                floor = FLOOR_NUM_to_ID[site_id][floor_id]

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                with open(info_path) as info_file:
                    info_data = json.load(info_file)             

                map_width = info_data["map_info"]["width"]
                map_height = info_data["map_info"]["height"]

                fig_steps = visualize_trajectory(trajectory=positions, is_multi = True,
                                                 floor_plan_filename=map_path, mode="lines + markers", title=f"{site_id}_{floor}_{sufix}", legends=legends, 
                                                 width_meter=map_width,  height_meter=map_height)
                save_figure_to_image(fig_steps, f"./img_out/predictions/{model_name}/{site_id}/{floor}_{sufix}.png")
            except:
                print(f"Exception: wrong floor-{floor} site-{site_id}")

        #break  # only first site_id
 

In [4]:
       
def snap2grid(predicted_data, grid_siteid_floorid, aux_grid_siteid_floorid, timestamps_traceid, snap_range=5, aux_snap_range=30):

    def closest_point(path, point, snap_range):
        #print(path)
        distance = (path[:, 0] - point[0])**2 + (path[:, 1] - point[1])**2
        
        if distance.min() < snap_range**2:
            idx = distance.argmin()
            return [path[idx, 0], path[idx, 1]], True
        else:
            return [point[0], point[1]], False

    snap2grid_data = {}

    n_s= 0
    for site_id in tqdm(predicted_data.keys()):  # over sites
        n_s += 1

        snap2grid_data[site_id] = {}
        for trace_id in predicted_data[site_id].keys():  # over traces

            trace = []  # list of points [x,y]
            predicted_record = predicted_data[site_id][trace_id].to_numpy()
            
            floor_id = int(np.median(predicted_record[:, 2]))
            grid = grid_siteid_floorid[site_id][floor_id]
            aux_grid = aux_grid_siteid_floorid[site_id][floor_id]
            
            func_x = interp1d(predicted_record[:, 3], predicted_record[:, 0], kind="linear", copy=False, fill_value="extrapolate")
            func_y = interp1d(predicted_record[:, 3], predicted_record[:, 1], kind="linear", copy=False, fill_value="extrapolate")

            _x = func_x(timestamps_traceid[trace_id])
            _y = func_y(timestamps_traceid[trace_id])

            for i, _ in enumerate(_x):  # over points 
                point = [_x[i], _y[i]]
                _closest_point, _isOk = closest_point(grid, point, snap_range)
                if _isOk:
                    trace.append(_closest_point)  # grid = closest step/waypoint point(slow) vs path_a0= closest contour point (fast)
                else:
                    _aus_close_point, _ = closest_point(aux_grid, point, aux_snap_range)
                    trace.append(_aus_close_point)
                

            snap2grid_data[site_id][trace_id] = pd.DataFrame(trace, columns=["x", "y"])
            snap2grid_data[site_id][trace_id]["floor"] = floor_id
            snap2grid_data[site_id][trace_id]["time"] = timestamps_traceid[trace_id]
            
    return snap2grid_data

def snap2grid_full(predicted_data, grid_siteid_floorid, snap_range=5):
    
    
    def sma(ar, length=1):

        if not isinstance(ar, np.ndarray): ar = ar.to_numpy()
        _length = min(length if length > 0 else 1, ar.size)  # length check
        if _length == 1:
            return ar
        else:
            _sum = np.cumsum(ar)
            _sum[_length:] = _sum[_length:] - _sum[:-_length]
            _sma = sma(ar[:_length - 1], _length-1)
            return np.concatenate((_sma, _sum[_length - 1:] / _length))

    def closest_point(path, point, snap_range):
        #print(path)
        distance = (path[:, 0] - point[0])**2 + (path[:, 1] - point[1])**2
        
        if distance.min() < snap_range**2:
            idx = distance.argmin()
            return [path[idx, 0], path[idx, 1]], True
        else:
            return [point[0], point[1]], False

    snap2grid_data = {}

    n_s= 0
    for site_id in tqdm(predicted_data.keys()):  # over sites
        n_s += 1

        snap2grid_data[site_id] = {}
        for trace_id in predicted_data[site_id].keys():  # over traces

            trace = []  # list of points [x,y]
            predicted_record = predicted_data[site_id][trace_id].to_numpy()
            
            floor_id = int(np.median(predicted_record[:, 2]))
            grid = grid_siteid_floorid[site_id][floor_id]

            for i, _ in enumerate(predicted_record):  # over points 
                point = [predicted_record[i, 0], predicted_record[i, 1]]
                _closest_point, _isOk = closest_point(grid, point, snap_range)
                if _isOk:
                    trace.append(_closest_point)  # grid = closest step/waypoint point(slow) vs path_a0= closest contour point (fast)
                else:
                    trace.append(point)
            trace = np.array(trace)
            trace[:, 0] = sma(trace[:, 0], 4)
            trace[:, 1] = sma(trace[:, 1], 4)

            snap2grid_data[site_id][trace_id] = pd.DataFrame(trace, columns=["x", "y"])
            snap2grid_data[site_id][trace_id]["floor"] = floor_id
            snap2grid_data[site_id][trace_id]["time"] = predicted_record[:, 3]
            
    return snap2grid_data

---------------------------------------------------------

## Combine/Plot/Submit blended models

In [20]:
num_blend = 12

blend_suffix = f"_post_x{num_blend}_1705"
model_name = f"blend{blend_suffix}"
grid_data = pickle.load(open("./data_out/freespace_05m_siteid_floorid.pkl", "rb"))

blend_data_median = blend_predictions_x(f"./submit/fit_data/post/blend_x{num_blend}_1705/", "blend_x7_0305_snapped2motion4s-fsV2nlB-02-04-02-02.pkl", grid_data, is_mean=False)
blend_data_mean = blend_predictions_x(f"./submit/fit_data/post/blend_x{num_blend}_1705/", "blend_x7_0305_snapped2motion4s-fsV2nlB-02-04-02-02.pkl", grid_data, is_mean=True)

with open(f"./submit/fit_data/post/{model_name}_median.pkl", "wb") as f:
    pickle.dump(blend_data_median, f)
    
with open(f"./submit/fit_data/post/{model_name}_mean.pkl", "wb") as f:
    pickle.dump(blend_data_mean, f)

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [21]:
plot_predictions_multi(f"{model_name}", blend_data_median, sufix=f"median")
plot_predictions_multi(f"{model_name}", blend_data_mean, sufix=f"mean")

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [22]:
predicted_data_post_median = pickle.load(open(f"./submit/fit_data/post/{model_name}_median.pkl", "rb")) #
predicted_data_post_mean = pickle.load(open(f"./submit/fit_data/post/{model_name}_mean.pkl", "rb")) #

grid_data = pickle.load(open("./data_out/waypoints_siteid_floorid.pkl", "rb"))
timestamps = pickle.load(open(f"./data_out/submission_timestamps_traceid.pkl", "rb"))

aux_grid_data = pickle.load(open("./data_out/freespace_1m_siteid_floorid.pkl", "rb"))

snapped2grid_median = snap2grid(predicted_data_post_median, grid_data, aux_grid_data, timestamps, snap_range=3, aux_snap_range=50)
snapped2grid_mean = snap2grid(predicted_data_post_mean, grid_data, aux_grid_data, timestamps, snap_range=3, aux_snap_range=50)
make_submission(f"{model_name}_median", snapped2grid_median, sufix=f"gridWPs3-gridFS1m50")
make_submission(f"{model_name}_mean", snapped2grid_mean, sufix=f"gridWPs3-gridFS1m50")
plot_predictions_multi(f"{model_name}", snapped2grid_median, sufix=f"median_gridWPs3-gridFS1m50")
plot_predictions_multi(f"{model_name}", snapped2grid_mean, sufix=f"mean_gridWPs3-gridFS1m50")

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [23]:
aux_grid_data = pickle.load(open("./data_out/freespace_2m_siteid_floorid.pkl", "rb"))

snapped2grid_median = snap2grid(predicted_data_post_median, grid_data, aux_grid_data, timestamps, snap_range=3, aux_snap_range=50)
snapped2grid_mean = snap2grid(predicted_data_post_mean, grid_data, aux_grid_data, timestamps, snap_range=3, aux_snap_range=50)
make_submission(f"{model_name}_median", snapped2grid_median, sufix=f"gridWPs3-gridFS2m50")
make_submission(f"{model_name}_mean", snapped2grid_mean, sufix=f"gridWPs3-gridFS2m50")
plot_predictions_multi(f"{model_name}", snapped2grid_median, sufix=f"median_gridWPs3-gridFS2m50")
plot_predictions_multi(f"{model_name}", snapped2grid_mean, sufix=f"mean_gridWPs3-gridFS2m50")

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]